# Machine Learning Model Experiments

Based on the features:
1. Classify what type of crime it is
2. Regression prediction on where the crime took place
3. Regression prediction on when the crime took place

Try out these models:
1) K Nearest Neighbors
2) Naive Bayesian Inference
3) Decision Tree
4) Random Forest
5) Logistic Regression
6) Support Vector Machine
7) Multi Layer Perceptron

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from datetime import datetime
import matplotlib.pyplot as plt

# Classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPClassifier, MLPRegressor

# Regressors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from xgboost import XGBClassifier, XGBRegressor

In [ ]:
data = pd.read_csv("data\data_full_features.csv")

In [ ]:
data_pipeline = ColumnTransformer([
    ("numerical", StandardScaler(), ["point_x", "point_y", "distance_to_nearest_police_station", 
                                     "temperature_2m_max (°F)", "temperature_2m_min (°F)", 
                                     "temperature_2m_mean (°F)", "precipitation_sum (mm)", "precipitation_hours (h)",
                                     "Unemployment Rate of a Population", "dispatch_dayOfWeek", "dispatch_day", 
                                     "dispatch_month", "dispatch_year", "block_number"]),
    ("categorical", OneHotEncoder(), ["crime_type"]),
    ("ordinal", OrdinalEncoder(), ["street_name"])
])

model_pipeline = Pipeline([
    ('data_preprocessing_pipeline', data_pipeline),
    ('model', )
])

model_pipeline.fit(X_train, y_train)
y_pred = model_pipeline.predict(X_test)